In [ ]:
#VGGNET16

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import time

# Define transformations for the training and testing data
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 training and test sets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Specify classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define a function to calculate accuracy
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Training Loop
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vggnet(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time
    train_accuracy = calculate_accuracy(trainloader, vggnet)
    test_accuracy = calculate_accuracy(testloader, vggnet)

    # Print statistics
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.3f}, "
          f"Training Accuracy: {train_accuracy:.2f}%, "
          f"Test Accuracy: {test_accuracy:.2f}%, "
          f"Training Time: {training_time:.2f} seconds")

print('Finished Training')


100%|██████████| 170498071/170498071 [00:05<00:00, 30874425.99it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch 1, Loss: 1.489, Training Accuracy: 53.50%, Test Accuracy: 52.66%, Training Time: 46.28 seconds
Epoch 2, Loss: 1.425, Training Accuracy: 55.21%, Test Accuracy: 53.40%, Training Time: 45.05 seconds
Epoch 3, Loss: 1.373, Training Accuracy: 54.64%, Test Accuracy: 52.82%, Training Time: 45.06 seconds
Epoch 4, Loss: 1.343, Training Accuracy: 58.43%, Test Accuracy: 55.72%, Training Time: 45.00 seconds
Epoch 5, Loss: 1.314, Training Accuracy: 59.20%, Test Accuracy: 56.56%, Training Time: 44.79 seconds
Finished Training


In [3]:
# GoogLeNet

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import time
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# Define transformations for the training and testing data
# GoogleNet expects a smaller input size than VGG, so we resize images to 224x224
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load CIFAR-10 training and test sets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Load pretrained GoogleNet Model
googlenet = models.googlenet(pretrained=True)

# Freeze model parameters
for param in googlenet.parameters():
    param.requires_grad = False

# Replace the last fully connected layer with a new one that has the same number of input features
# but 10 output features for CIFAR-10
num_ftrs = googlenet.fc.in_features
googlenet.fc = nn.Linear(num_ftrs, 10)

# Convert model to be used on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
googlenet = googlenet.to(device)

# Define Optimizer and Loss Function
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(googlenet.fc.parameters(), lr=0.001)

# Define a function to calculate accuracy
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Training Loop
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = googlenet(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time
    train_accuracy = calculate_accuracy(trainloader, googlenet)
    test_accuracy = calculate_accuracy(testloader, googlenet)

    # Print statistics
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.3f}, "
          f"Training Accuracy: {train_accuracy:.2f}%, "
          f"Test Accuracy: {test_accuracy:.2f}%, "
          f"Training Time: {training_time:.2f} seconds")

print('Finished Training')


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 56.9MB/s]


Epoch 1, Loss: 0.894, Training Accuracy: 76.48%, Test Accuracy: 75.15%, Training Time: 98.28 seconds
Epoch 2, Loss: 0.676, Training Accuracy: 78.11%, Test Accuracy: 76.60%, Training Time: 98.98 seconds
Epoch 3, Loss: 0.645, Training Accuracy: 78.52%, Test Accuracy: 77.50%, Training Time: 99.12 seconds
Epoch 4, Loss: 0.635, Training Accuracy: 78.14%, Test Accuracy: 77.04%, Training Time: 97.44 seconds
Epoch 5, Loss: 0.629, Training Accuracy: 78.70%, Test Accuracy: 77.28%, Training Time: 97.17 seconds
Finished Training


In [5]:
#Resnet50

In [7]:
# Load pretrained ResNet-50 Model instead of ResNet-18
resnet50 = models.resnet50(pretrained=True)

# Freeze model parameters
for param in resnet50.parameters():
    param.requires_grad = False

# Replace the fully connected layer with a new one with 10 output features for CIFAR-10
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 10)

# Convert model to be used on GPU
resnet50 = resnet50.to(device)

# Update the optimizer to optimize the parameters of resnet50
optimizer = optim.Adam(resnet50.fc.parameters(), lr=0.001)

# Training Loop
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    start_time = time.time()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet50(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time
    train_accuracy = calculate_accuracy(trainloader, resnet50)
    test_accuracy = calculate_accuracy(testloader, resnet50)

    # Print statistics
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.3f}, "
          f"Training Accuracy: {train_accuracy:.2f}%, "
          f"Test Accuracy: {test_accuracy:.2f}%, "
          f"Training Time: {training_time:.2f} seconds")

print('Finished Training')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 60.4MB/s]


Epoch 1, Loss: 1.642, Training Accuracy: 45.97%, Test Accuracy: 45.02%, Training Time: 20.32 seconds
Epoch 2, Loss: 1.492, Training Accuracy: 51.37%, Test Accuracy: 49.67%, Training Time: 19.96 seconds
Epoch 3, Loss: 1.438, Training Accuracy: 51.19%, Test Accuracy: 49.48%, Training Time: 20.40 seconds
Epoch 4, Loss: 1.409, Training Accuracy: 52.78%, Test Accuracy: 50.81%, Training Time: 19.62 seconds
Epoch 5, Loss: 1.399, Training Accuracy: 53.18%, Test Accuracy: 51.38%, Training Time: 20.37 seconds
Finished Training
